In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D,Conv2DTranspose,UpSampling2D
from keras import regularizers, optimizers,Sequential
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import keras
from sklearn.metrics import recall_score
from tensorflow.python.keras import backend as K 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import os


from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model,load_model
from keras.optimizers import Adam
from keras.applications.inception_resnet_v2 import InceptionResNetV2
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        #print(os.path.join(dirname, filename))
#        print(dirname)
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
os.listdir('/kaggle/')

['lib', 'input', 'config', 'working']

In [3]:
os.listdir('/kaggle/input')

['multi-weights',
 'inceptionresnetv2',
 'models-weights',
 'segmentationmodels',
 'tta-git',
 'severstal-steel-defect-detection']

In [4]:
os.listdir('/kaggle/input/severstal-steel-defect-detection')

['sample_submission.csv', 'train.csv', 'train_images', 'test_images']

In [5]:
os.listdir('/kaggle/input/severstal-steel-defect-detection/')
dir1='/kaggle/input/severstal-steel-defect-detection/'

In [6]:
df=pd.read_csv(dir1+'train.csv')

In [7]:
sub=pd.read_csv(dir1+'sample_submission.csv')
sub['imageid']=sub.ImageId_ClassId.apply(lambda x: x.split('_')[0])
sub['class']=sub.ImageId_ClassId.apply(lambda x: int(x.split('_')[1]))
sub['EncodedPixels']=" "
print(sub.shape)
sub.head()


(7204, 4)


,ImageId_ClassId,EncodedPixels,imageid,class
0,004f40c73.jpg_1,,004f40c73.jpg,1
1,004f40c73.jpg_2,,004f40c73.jpg,2
2,004f40c73.jpg_3,,004f40c73.jpg,3
3,004f40c73.jpg_4,,004f40c73.jpg,4
4,006f39c41.jpg_1,,006f39c41.jpg,1


In [8]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=16, dim=(128, 800),preprocess=None, n_channels=3,
                 n_classes=1, random_state=2019, shuffle=False):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.preprocess = preprocess
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.on_epoch_end()
    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(indexes)
        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
            
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            #print(i,ID)
            im_name = self.df['imageid'][ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_rgb(img_path)
            #print(im_name,img_path)
            # Store samples
            img = cv2.resize(img,(self.dim[1],self.dim[0]))
            X[i,] = img 
            #print(" X sahpe",X.shape)
            #print(" img sahpe",img.shape)
            # normalize 
            #X = X / 255
        if self.preprocess!=None: X = self.preprocess(X)

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['imageid'][ID]
            #image_df = self.target_df[self.target_df['imageid'] == im_name]
            
            rles = self.df['EncodedPixels'][ID]
            h,w=self.dim
            masks = rle_to_mask(rles, 256,1600)
            masks = cv2.resize(masks,(self.dim[1],self.dim[0]))

            #print(" y sahpe",y.shape)
            #print(" masks sahpe",masks.shape)
            y[i, ] = np.expand_dims(masks, -1)
            y = (y > 0).astype(int)
        return y 

        
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img

In [9]:
augmentation_gen=ImageDataGenerator(rescale=1./255.,
                          shear_range=0.1,
                          rotation_range=60,
                          brightness_range=[0.6,1.0],
                          zoom_range=0.1,
                          channel_shift_range=0.1,
                          horizontal_flip=True,
                          vertical_flip=True,
                          )

In [10]:
from keras import backend as K
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
#Recall shall be the model metric we use to select our best model when there is a high cost associated with False Negative.
#TPR
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

In [11]:
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

In [12]:
sub0=sub[['imageid','class']][sub['class'].astype(int)==1]

print("shape of df:",sub0.shape)
sub0.head()

shape of df: (1801, 2)


,imageid,class
0,004f40c73.jpg,1
4,006f39c41.jpg,1
8,00b7fb703.jpg,1
12,00bbcd9af.jpg,1
16,0108ce457.jpg,1


In [13]:
model = InceptionResNetV2(weights=None, input_shape=(299,299,3), include_top=False)

In [14]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(64,activation='relu')(x) 
out=Dense(4,activation='sigmoid')(x) #final layer 5 clases

model_m=Model(inputs=model.input,outputs=out)

In [15]:
model_m.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy',precision,recall])
#model_m.summary()

In [16]:
model_m.load_weights('/kaggle/input/multi-weights/only_multilabel_weights.h5')

In [17]:
bs=1
test_gen_mul = DataGenerator(
        sub0.index,
        df=sub0,
        dim=(299,299),
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=bs,
        n_classes=1)

In [18]:
#TTA
tta_steps = 6
multi_class= []
for val in tqdm_notebook(test_gen_mul):
    
    batch_pred = []
    for i in range(tta_steps):
        preds = model_m.predict_generator(augmentation_gen.flow(val,batch_size=bs, shuffle=False),steps=1)
        batch_pred.append(preds)
    pred = np.mean(batch_pred, axis=0)
    multi_class.append(pred)

In [19]:
import gc
def clear(var):
    del var
    return gc.collect()

In [20]:
clear(test_gen_mul),clear(model_m)

(120, 0)

In [21]:
multiC=[np.array(v).flatten().tolist() for v in multi_class]
arr=np.array(multiC)
multi_frame=pd.DataFrame(arr,columns=['class1','class2','class3','class4'])
multi_frame.index=sub0.index
df_mul=sub0.join(multi_frame)
df_mul['class1']=df_mul['class1'].apply(lambda v:  1 if v >0.5 else 0)
df_mul['class2']=df_mul['class2'].apply(lambda v:  1 if v >0.5 else 0)
df_mul['class3']=df_mul['class3'].apply(lambda v:  1 if v >0.5 else 0)
df_mul['class4']=df_mul['class4'].apply(lambda v:  1 if v >0.5 else 0)
df_mul.head()

,imageid,class,class1,class2,class3,class4
0,004f40c73.jpg,1,1,0,0,0
4,006f39c41.jpg,1,0,0,1,0
8,00b7fb703.jpg,1,0,0,1,0
12,00bbcd9af.jpg,1,0,0,1,0
16,0108ce457.jpg,1,0,0,1,0


In [22]:
c1=df_mul[df_mul['class1']==1]
c2=df_mul[df_mul['class2']==1]
c3=df_mul[df_mul['class3']==1]
c4=df_mul[df_mul['class4']==1]

In [23]:
import sys
sys.path.append("../input/tta-git/")

In [24]:
from tta_wrapper import tta_segmentation

 ##  Multi Label Classification

## Mask Prediction

In [25]:
def mask_to_rle(mask):
    '''
    Convert a mask into RLE
    
    Parameters: 
    mask (numpy.array): binary mask of numpy array where 1 - mask, 0 - background

    Returns: 
    sring: run length encoding 
    '''
    pixels= mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [26]:
def rle_to_mask(rle_string, height, width):
    
    rows, cols = height, width
    img = np.zeros(rows * cols, dtype=np.uint8)
    if len(str(rle_string)) > 1:
        rle_numbers = [int(numstring) for numstring in rle_string.split(' ')]
        rle_pairs = np.array(rle_numbers).reshape(-1, 2)
        for index, length in rle_pairs:
            index -= 1
            img[index:index+length] = 255
    else: img = np.zeros(cols*rows)
    img = img.reshape(cols, rows)
    img = img.T
    return img

In [27]:
'metric and loss function for evaluation'
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def loss_dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return -K.log((2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

### Class1

In [28]:
import sys
sys.path.append("../input/tta-git/")


In [29]:
from tta_wrapper import tta_segmentation


In [30]:
mask_model=load_model('/kaggle/input/models-weights/mod_c1.h5',custom_objects={"loss_dice_coef":loss_dice_coef,"dice_coef":dice_coef})

In [31]:
pred_c1=Model(inputs=mask_model.input,outputs=mask_model.output)
pred_c1.load_weights('/kaggle/input/models-weights/c1_weights')


In [32]:
tta_c1 = tta_segmentation(pred_c1, h_flip=True,v_flip=True, h_shift=(-10, 10), merge='mean')

In [33]:
bs=1
genc1 = DataGenerator(
        c1.index,
        df=c1,
        dim=(128,800),
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=bs,
        n_classes=1)


In [34]:
pred_mask1=tta_c1.predict_generator(genc1)

In [35]:
for i,j in enumerate(c1.index):
    img=np.squeeze(pred_mask1[i])
    img = cv2.resize(img, (1600, 256))
    tmp = np.copy(img)
    tmp[tmp<0.5] = 0
    tmp[tmp>0] = 1
    pred_rle=mask_to_rle(tmp)
    sub["EncodedPixels"][j]=pred_rle

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
clear(genc1),clear(tta_c1),clear(pred_c1)

(0, 0, 0)

### Class2

In [ ]:
pred_c2=Model(inputs=mask_model.input,outputs=mask_model.output)
pred_c2.load_weights('/kaggle/input/models-weights/c2_weights')
tta_c2 = tta_segmentation(pred_c2, h_flip=True,v_flip=True, h_shift=(-10, 10), merge='mean')


In [ ]:
bs=1
genc2 = DataGenerator(
        c2.index,
        df=c2,
        dim=(128,800),
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=bs,
        n_classes=1)
 

In [ ]:
pred_mask2=tta_c2.predict_generator(genc2) 

In [ ]:
for i,j in enumerate(c2.index):
    img=np.squeeze(pred_mask2[i])
    img = cv2.resize(img, (1600, 256))
    tmp = np.copy(img)
    tmp[tmp<0.5] = 0
    tmp[tmp>0] = 1
    pred_rle=mask_to_rle(tmp)
    sub["EncodedPixels"][j]=pred_rle

In [ ]:
clear(genc2),clear(pred_c2),clear(tta_c2)

### Class3

In [ ]:
pred_c3=Model(inputs=mask_model.input,outputs=mask_model.output)
pred_c3.load_weights('/kaggle/input/models-weights/c3_weights')
tta_c3 = tta_segmentation(pred_c3, h_flip=True,v_flip=True, h_shift=(-10, 10), merge='mean')


In [ ]:
bs=1
genc3 = DataGenerator(
        c3.index,
        df=c3,
        dim=(128,800),
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=bs,
        n_classes=1)
 
pred_mask3 = tta_c3.predict_generator(genc3)


In [ ]:
for i,j in enumerate( c3.index):
    img=np.squeeze(pred_mask3[i])
    img = cv2.resize(img, (1600, 256))
    tmp = np.copy(img)
    tmp[tmp<0.5] = 0
    tmp[tmp>0] = 1
    pred_rle=mask_to_rle(tmp)
    sub["EncodedPixels"][j]=pred_rle

In [ ]:
clear(genc3),clear(pred_c3),clear(tta_c3)

### Class4

In [ ]:
pred_c4=Model(inputs=mask_model.input,outputs=mask_model.output)
pred_c4.load_weights('/kaggle/input/models-weights/c4_weights')
tta_c4 = tta_segmentation(pred_c4, h_flip=True,v_flip=True, h_shift=(-10, 10), merge='mean')

In [ ]:
bs=1
genc4 = DataGenerator(
        c4.index,
        df=c4,
        dim=(128,800),
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=bs,
        n_classes=1)
 
pred_mask4 = tta_c4.predict_generator(genc4)


In [ ]:
for i,j in enumerate(c4.index):
    img=np.squeeze(pred_mask4[i])
    img = cv2.resize(img, (1600, 256))
    tmp = np.copy(img)
    tmp[tmp<0.5] = 0
    tmp[tmp>0] = 1
    pred_rle=mask_to_rle(tmp)
    sub["EncodedPixels"][j]=pred_rle

In [ ]:
"""f, axarr = plt.subplots(2,2,figsize=(7, 7))
axarr[0,0].imshow(np.squeeze(test_preds[k] > 0.5).astype(int))
axarr[0,1].imshow(np.squeeze(test_preds2[k] > 0.5).astype(int))
axarr[1,0].imshow(np.squeeze(test_preds3[k] > 0.5).astype(int))
axarr[1,1].imshow(np.squeeze(test_preds4[k] > 0.5).astype(int))
"""

In [ ]:
"""from IPython.display import FileLink
#FileLink(r'df_name.csv')
model.save(r"steel_multi_model.h5")
FileLink(r"steel_multi_model.h5")"""

In [ ]:
"steel_binary_model.h5"

In [ ]:
sub[['ImageId_ClassId','EncodedPixels']].to_csv('submission.csv', index=False)